# Project 009 - Transfer Learning with RESNET50 -  Keras Implementation


In [33]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, resnet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [34]:
#(You can swap this out for your own dataset later.)
#Save CIFAR-10 images to folders

import os
import cv2
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

classes = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

def save_split(split_name, images, labels):
    base = f"cifar10_{split_name}"
    os.makedirs(base, exist_ok=True)

    for i, (img, lbl) in enumerate(zip(images, labels)):
        cls = classes[lbl[0]]
        cls_path = os.path.join(base, cls)
        os.makedirs(cls_path, exist_ok=True)
        cv2.imwrite(os.path.join(cls_path, f"{i}.png"), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

save_split("train", x_train, y_train)
save_split("test", x_test, y_test)

In [35]:
#Use a generator that automatically resizes to 224×224

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(
    "cifar10_train",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    "cifar10_test",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [36]:
#Apply ResNet50 Preprocessing

x_train = resnet50.preprocess_input(x_train)
x_test = resnet50.preprocess_input(x_test)

In [37]:
#Load Pre-trained ResNet50 (without top layer)

base_model = ResNet50(
    weights="imagenet",
    include_top=False, #this is to help us change the 1000 classes from ImageNet for us to add our own
    input_shape=(224, 224, 3) #this defines the expected input to be feed into the ResNet model
)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

In [38]:
#Add Custom Classification Head

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

In [39]:
#Compile the Model

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [40]:
#Train the Classification Head Only (Feature Extraction)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 192s 113ms/step - accuracy: 0.6206 - loss: 1.1014 - val_accuracy: 0.7629 - val_loss: 0.6699
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 162s 104ms/step - accuracy: 0.7509 - loss: 0.7201 - val_accuracy: 0.8006 - val_loss: 0.5632
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 166s 106ms/step - accuracy: 0.7674 - loss: 0.6746 - val_accuracy: 0.8000 - val_loss: 0.5554
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 163s 104ms/step - accuracy: 0.7828 - loss: 0.6311 - val_accuracy: 0.8050 - val_loss: 0.5620
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 163s 104ms/step - accuracy: 0.7935 - loss: 0.5943 - val_accuracy: 0.8156 - val_loss: 0.5237
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 164s 105ms/step - accuracy: 0.8031 - loss: 0.5695 - val_accuracy: 0.8123 - val_loss: 0.5256
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 169s 108ms/step - accuracy: 0.8076 - loss: 0.5489 - val_accuracy: 0.8229 - val_loss: 0.5074
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 165s 106ms/step - accuracy: 0.8

In [43]:
#Fine-Tune the Deeper ResNet Layers (Optional but increases accuracy)

for layer in base_model.layers[-30:]:   # Unfreeze last 30 layers
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5
)


Epoch 1/5
  50/1563 ━━━━━━━━━━━━━━━━━━━━ 3:04 122ms/step - accuracy: 0.6099 - loss: 1.2424

KeyboardInterrupt: 

In [ ]:
#Evaluate

loss, acc = model.evaluate(x_test, y_test)
print("Final Test Accuracy:", acc)

#Predict on an External Image

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_external(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)

    img_array = np.expand_dims(img_array, axis=0)
    img_array = resnet50.preprocess_input(img_array)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)

    print("Predicted Class:", predicted_class)

    plt.imshow(image.load_img(img_path))
    plt.title("Predicted: " + str(predicted_class))
    plt.axis("off")
    plt.show()

predict_external("014.png")
